In [ ]:
!wget https://www.robots.ox.ac.uk/~vgg/data/oxbuildings/oxbuild_images.tgz

--2021-06-08 13:17:54--  https://www.robots.ox.ac.uk/~vgg/data/oxbuildings/oxbuild_images.tgz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1980280437 (1.8G) [application/x-gzip]
Saving to: ‘oxbuild_images.tgz’

oxbuild_images.tgz  100%[===================>]   1.84G  20.6MB/s    in 91s     

2021-06-08 13:19:27 (20.7 MB/s) - ‘oxbuild_images.tgz’ saved [1980280437/1980280437]



In [ ]:
!tar -xvzf oxbuild_images.tgz

In [ ]:
! mkdir userImage

In [ ]:
pip install fastdist


In [ ]:
import cv2
import os , glob
import math
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial
from fastdist import fastdist
from IPython.display import Image as image1 , display
from sklearn.manifold import MDS



In [ ]:
def makeHistogram(image):
    redHistogram, bin_edges = np.histogram(image[:,:,0], bins=256, range=(0,256),density=True)
    greenHistogram, bin_edges = np.histogram(image[:,:,1], bins=256, range=(0,256),density=True)
    blueHistogram , bin_edges = np.histogram(image[:,:,2], bins=256, range=(0,256),density=True)
    histogram = np.hstack((redHistogram ,greenHistogram , blueHistogram ))
    return histogram


In [ ]:
filenames = [img for img in glob.glob("*.jpg")]

filenames.sort() # ADD THIS LINE
totalHistograms = []
for img in filenames:
    image= cv2.imread(img)
    #--------------------make histogram-------------------------
    histogram = makeHistogram(image)
    totalHistograms.append(histogram)
    # print (img)
    
len(filenames)

5063

In [ ]:
totalHistograms = np.array(totalHistograms)
histogram.shape
totalHistograms.shape

(5063, 768)

In [ ]:

# def showImage(arr):
#   img1 = np.flip(arr, 2)
#   img = Image.fromarray(img1)
#   img.save('my2.png')
#   display(image1('my2.png'))

# showImage(images[2])


In [ ]:

# totalHistograms = []
# for image in images:
#   redHistogram, bin_edges = np.histogram(image[:,:,0], bins=32, range=(0,256),density=True)
#   greenHistogram, bin_edges = np.histogram(image[:,:,1], bins=32, range=(0,256),density=True)
#   blueHistogram , bin_edges = np.histogram(image[:,:,2], bins=32, range=(0,256),density=True)
#   histogram = np.hstack((redHistogram ,greenHistogram , blueHistogram ))
#   totalHistograms.append(histogram)
# totalHistograms = np.array(totalHistograms)
# histogram.shape
# totalHistograms.shape
# # redHistogram

(1001, 96)

In [ ]:
distanceMode = 0

In [ ]:
DistanceMatrix = np.zeros((totalHistograms.shape[0], totalHistograms.shape[0]))

def euclideanDistance(totalHistograms):
  for i in range(totalHistograms.shape[0]):
      for j in range(totalHistograms.shape[0]):
        DistanceMatrix[i,j]=fastdist.euclidean(totalHistograms[i] , totalHistograms[j])
        
# euclideanDistance(totalHistograms)
# euclideanDistanceMatrix.shape

In [ ]:

def cosineDistance(totalHistograms):
  for i in range(totalHistograms.shape[0]):
      for j in range(totalHistograms.shape[0]):
        DistanceMatrix[i,j]= 1 - fastdist.cosine(totalHistograms[i],totalHistograms[j] )      

# cosineDistance(totalHistograms)
# cosineDistanceMatrix.shape


In [ ]:

def manhattanDistance(totalHistograms):
  for i in range(totalHistograms.shape[0]):
      for j in range(totalHistograms.shape[0]):
        DistanceMatrix[i,j]= fastdist.cityblock(totalHistograms[i],totalHistograms[j])      

# manhattanDistance(totalHistograms)
# manhattanDistanceMatrix.shape


In [ ]:
def makeDistance(distanceMode):
  if (distanceMode == 0):
    euclideanDistanceDistance(totalHistograms)
  if (distanceMode == 1):
    cosineDistance(totalHistograms)
  if (distanceMode == 2):
    manhattanDistance(totalHistograms)


In [ ]:

DistanceMatrix.shape

euclidean ==> 0 |  cosine ==> 1 | manhattan ==> 2   1


0.836850331122533

In [ ]:
def findMostDiffrentImage(topImages , index ,distance):
   indexOfMaxDistance =  topImages[1].argmax(axis=0)
   if(distance <= topImages[1][indexOfMaxDistance]):
     topImages[1][indexOfMaxDistance] =  distance
     topImages[0][indexOfMaxDistance] =  index





In [ ]:
distanceMode = input("euclidean ==> 0 |  cosine ==> 1 | manhattan ==> 2   ")
makeDistance(int(distanceMode))
embedding = MDS(n_components=10)
V = embedding.fit_transform(DistanceMatrix)
V.shape


euclidean ==> 0 |  cosine ==> 1 | manhattan ==> 2   1


/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_mds.py:419: UserWarning: The MDS API has changed. ``fit`` now constructs an dissimilarity matrix from data. To use a custom dissimilarity matrix, set ``dissimilarity='precomputed'``.
  warnings.warn("The MDS API has changed. ``fit`` now constructs an"


(5063, 10)

In [72]:
inputType = int(input("existing image ==> 0 |  new image ==> 1     "))
distanceMode1 = int(input("euclidean ==> 0 |  cosine ==> 1 | manhattan ==> 2    "))
NUMBER_OF_TOP_IMAGES = 20
topImages = np.ones((2, NUMBER_OF_TOP_IMAGES)) * np.inf

if (inputType == 0):
  userInput = int(input("enter number of image    "))
  for i in range(0,V.shape[0]):
    if userInput == i :
      continue
    else:
      if (distanceMode1 == 0):
         idist = fastdist.euclidean(V[i],V[userInput] )
      if (distanceMode1 == 1):
         idist =1 - fastdist.cosine(V[i],V[userInput] )
      if (distanceMode1 == 2):
         idist =fastdist.cityblock(V[i],V[userInput] )
      findMostDiffrentImage(topImages , i , idist)
if (inputType == 1):
    userImageName = input("enter name of image :     ")
    image= cv2.imread("userImage/"+userImageName)
    userImageHistogram = makeHistogram(image)
    for i in range(0,totalHistograms.shape[0]):
      if (distanceMode1 == 0):
         idist = fastdist.euclidean(userImageHistogram , totalHistograms[i] )
      if (distanceMode1 == 1):
         idist = 1 - fastdist.cosine(userImageHistogram , totalHistograms[i] )
      if (distanceMode1 == 2):
         idist =fastdist.cityblock(userImageHistogram , totalHistograms[i] )
      findMostDiffrentImage(topImages , i , idist)

existing image ==> 0 |  new image ==> 1     0
euclidean ==> 0 |  cosine ==> 1 | manhattan ==> 2    0
enter number of image    4800


In [ ]:
if (inputType == 0):
    display(image1(filenames[userInput]))
if (inputType == 1):
    display(image1("userImage/" + userImageName))

In [ ]:
for i in range(0,NUMBER_OF_TOP_IMAGES):
  display(image1(filenames[int(topImages[0][i])]))

In [ ]:
topImages

array([[4.45900000e+03, 4.90000000e+03, 3.55100000e+03, 3.60000000e+02,
        4.17500000e+03, 3.51000000e+02, 4.46000000e+03, 1.71500000e+03,
        3.54500000e+03, 2.73000000e+03],
       [2.17724326e-01, 2.23757879e-01, 9.56925928e-02, 1.98246897e-01,
        1.91836464e-01, 1.39477229e-01, 2.21538423e-01, 2.17421525e-01,
        2.27761964e-01, 1.99408593e-01]])